In [1]:
import os

In [2]:
%pwd

'd:\\end-to-end-mlops\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\end-to-end-mlops'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import * 
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        print(f"Config file path: {config_filepath}")
        print(f"Params file path: {params_filepath}")
        print(f"Schema file path: {schema_filepath}")

        # Check if config files exist and handle missing files gracefully
        if not os.path.exists(config_filepath):
            raise FileNotFoundError(f"Config file not found at path: {config_filepath}")
        if not os.path.exists(params_filepath):
            raise FileNotFoundError(f"Params file not found at path: {params_filepath}")
        if not os.path.exists(schema_filepath):
            raise FileNotFoundError(f"Schema file not found at path: {schema_filepath}")

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted zip file to: {unzip_path}")

In [10]:
# Try block for error handling
try:
    # Initialize ConfigurationManager
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    # Data ingestion process
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except FileNotFoundError as fnf_error:
    logger.error(f"File not found error: {fnf_error}")
except Exception as e:
    logger.error(f"An error occurred: {e}")
    raise e

Config file path: config\config.yaml
Params file path: params.yaml
Schema file path: schema.yaml
[2024-10-04 09:26:07,379: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-04 09:26:07,379: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-04 09:26:07,379: ERROR: 718914265: An error occurred: yaml file is empty]


ValueError: yaml file is empty